In [1]:
import pandas as pd
import numpy as np

In [2]:
X_train = pd.read_csv("D:/Hackathons/GST_GOV/Train_60/X_Train_Data_Input_Imputed.csv")
y_train = pd.read_csv('D:/Hackathons/GST_GOV/Train_60/Y_Train_Data_Target.csv')
X_test = pd.read_csv('D:/Hackathons/GST_GOV/Test_20/X_Test_Data_Input_Imputed.csv')
y_test = pd.read_csv('D:/Hackathons/GST_GOV/Test_20/Y_Test_Data_Target.csv')

In [ ]:
os.makedirs('hypertuned_model_plots', exist_ok=True)
os.makedirs('hypertuned_model', exist_ok=True)
os.makedirs('optuna_visualizations/XGBoost', exist_ok=True)
os.makedirs('optuna_visualizations/LightGBM', exist_ok=True)
os.makedirs('optuna_visualizations/CatBoost', exist_ok=True)
os.makedirs('optuna_logs', exist_ok=True)

In [3]:
y_train = y_train.drop('ID', axis=1)
y_test = y_test.drop('ID', axis=1)

In [4]:
import optuna
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import gc
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, roc_curve, confusion_matrix, log_loss, balanced_accuracy_score)

from optuna.samplers import TPESampler
from optuna.visualization import (plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour,
                                  plot_param_importances, plot_intermediate_values)
import plotly.io as pio

In [5]:
hypertuned_models = {}

# Helper function to plot AUC-ROC curve with metrics
def plot_roc_curve(fpr, tpr, auc_roc, name, accuracy, precision, recall, f1):
    plt.figure()
    plt.plot(fpr, tpr, label=f'AUC-ROC (AUC = {auc_roc:.4f})')
    plt.plot([0, 1], [0, 1], 'k--', label='Random Guessing')
    plt.xlabel('False Positive Rate (FPR)')
    plt.ylabel('True Positive Rate (TPR)')
    plt.title(f'AUC-ROC Curve - {name}\n'
              f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.savefig(f'hypertuned_model_plots/{name}_roc_curve.png')
    plt.close()

# Helper function to plot confusion matrix with metrics
def plot_confusion_matrix(cm, name, accuracy, precision, recall, f1):
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'Confusion Matrix - {name}\n'
              f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.savefig(f'hypertuned_model_plots/{name}_confusion_matrix.png')
    plt.close()

# Helper function to save Optuna visualizations as interactive HTML files
def save_optuna_visualizations(study, model_name):
    viz_dir = f'optuna_visualizations/{model_name}'
    os.makedirs(viz_dir, exist_ok=True)
    
    # Save interactive visualizations
    opt_hist = plot_optimization_history(study)
    pio.write_html(opt_hist, f'{viz_dir}/{model_name}_optimization_history.html')
    
    parallel_coord = plot_parallel_coordinate(study)
    pio.write_html(parallel_coord, f'{viz_dir}/{model_name}_parallel_coordinate.html')
    
    slice_plot = plot_slice(study)
    pio.write_html(slice_plot, f'{viz_dir}/{model_name}_slice.html')
    
    contour_plot = plot_contour(study)
    pio.write_html(contour_plot, f'{viz_dir}/{model_name}_contour.html')
    
    param_importances = plot_param_importances(study)
    pio.write_html(param_importances, f'{viz_dir}/{model_name}_param_importances.html')

    # New: Intermediate values plot (visualizes objective value per step of trials)
    intermediate_values_plot = plot_intermediate_values(study)
    pio.write_html(intermediate_values_plot, f'{viz_dir}/{model_name}_intermediate_values.html')

# Optuna tuning and training
def tune_and_train_model(model_name, train_func, param_func):
    def objective(trial):
        params = param_func(trial)  # Define the search space
        clf = train_func(params)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        return accuracy  # Maximize accuracy in the objective function
    
    # Use TPESampler (efficient for hyperparameter optimization)
    sampler = TPESampler(seed=42)
    study = optuna.create_study(direction='maximize', sampler=sampler)
    
    # Run extensive tuning with 100 trials and no time constraint
    print(f"Starting tuning for {model_name}")
    study.optimize(objective, n_trials=50, timeout=None)
    
    # Get the best trial parameters
    best_params = study.best_params
    best_accuracy = study.best_value
    best_model = train_func(best_params)
    
    # Retrain the best model with best hyperparameters
    best_model.fit(X_train, y_train)
    
    # Predict and calculate metrics
    y_pred = best_model.predict(X_test)
    y_prob = best_model.predict_proba(X_test)[:, 1]
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc_roc = roc_auc_score(y_test, y_prob)
    cm = confusion_matrix(y_test, y_pred)
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    
    # Plot AUC-ROC and Confusion Matrix
    plot_roc_curve(fpr, tpr, auc_roc, model_name, accuracy, precision, recall, f1)
    plot_confusion_matrix(cm, model_name, accuracy, precision, recall, f1)
    
    # Save the best model using pickle
    model_path = f'hypertuned_models/{model_name}.pkl'
    with open(model_path, 'wb') as model_file:
        pickle.dump(best_model, model_file)
    
    # Store in dictionary
    hypertuned_models[model_name] = {
        'best_params': best_params,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_roc,
        'confusion_matrix': cm.tolist(),
        'fpr': fpr.tolist(),
        'tpr': tpr.tolist(),
        'thresholds': thresholds.tolist(),
        'model_pickle': model_path
    }
    
    # Log the study results to a file
    log_path = f'optuna_logs/{model_name}_optuna_log.txt'
    with open(log_path, 'w') as log_file:
        log_file.write(study.trials_dataframe().to_csv(index=False))
    
    # Save Optuna visualizations
    save_optuna_visualizations(study, model_name)
    
    print(f"{model_name} tuning complete. Best accuracy: {best_accuracy:.4f}")

In [6]:
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# Create directories for these models
os.makedirs('xgboost_models', exist_ok=True)
os.makedirs('lightgbm_models', exist_ok=True)
os.makedirs('catboost_models', exist_ok=True)


# Function to train XGBoost
def train_xgboost(params):
    return xgb.XGBClassifier(**params)

# Function to train LightGBM
def train_lightgbm(params):
    return lgb.LGBMClassifier(**params)

# Function to train CatBoost
def train_catboost(params):
    return cb.CatBoostClassifier(**params, silent=True)

# Define parameter space for each model
def xgboost_params(trial):
    return {
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
    }

def lightgbm_params(trial):
    return {
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'max_depth': trial.suggest_int('max_depth', -1, 50)
    }

def catboost_params(trial):
    return {
        'iterations': trial.suggest_int('iterations', 100, 2000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 10),
        'random_strength': trial.suggest_float('random_strength', 0, 10),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide'])
    }

# Optuna tuning for XGBoost


# Optuna tuning for LightGBM


# Optuna tuning for CatBoost


# Free memory
gc.collect()


0

In [7]:
tune_and_train_model('XGBoost', train_xgboost, xgboost_params)

[I 2024-10-08 06:56:18,017] A new study created in memory with name: no-name-a7805a00-896e-4b78-baec-170fe85963e3


Starting tuning for XGBoost


[I 2024-10-08 06:57:12,300] Trial 0 finished with value: 0.9774179250473803 and parameters: {'n_estimators': 812, 'max_depth': 15, 'learning_rate': 0.22227824312530747, 'colsample_bytree': 0.7190609389379257, 'subsample': 0.5780093202212182, 'gamma': 1.5599452033620265, 'reg_alpha': 0.5808361216819946, 'reg_lambda': 8.661761457749352, 'min_child_weight': 7}. Best is trial 0 with value: 0.9774179250473803.
[I 2024-10-08 06:58:38,045] Trial 1 finished with value: 0.978155376902855 and parameters: {'n_estimators': 1446, 'max_depth': 3, 'learning_rate': 0.29127385712697834, 'colsample_bytree': 0.8827098485602951, 'subsample': 0.6061695553391381, 'gamma': 1.8182496720710062, 'reg_alpha': 1.8340450985343382, 'reg_lambda': 3.0424224295953772, 'min_child_weight': 6}. Best is trial 1 with value: 0.978155376902855.
[I 2024-10-08 06:59:19,438] Trial 2 finished with value: 0.9780865990095984 and parameters: {'n_estimators': 921, 'max_depth': 6, 'learning_rate': 0.18743733946949004, 'colsample_bytr

XGBoost tuning complete. Best accuracy: 0.9786
